In [1]:
import os
import sys
sys.path.append('..')

In [2]:
import random
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [3]:
from source.data import dataLoader
from source.model import Model
from source.loss import Loss, Metric
from source.train import trainModel

### Seed

In [4]:
seed = 2017
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

### Model

In [5]:
model = Model()
model = model.to('cuda:0')

### Loss

In [6]:
loss = Loss()
metric = Metric() 

### Optimizer

In [7]:
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
schedular = ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

### Data

In [8]:
train = pd.read_csv('../../data/train.csv')
valid = pd.read_csv('../../data/valid.csv')
path = '../../data/'
train, valid = dataLoader(path, train, valid)
train = DataLoader(train, batch_size=4, shuffle=True, num_workers=1, drop_last=True)
valid = DataLoader(valid, batch_size=4, shuffle=True, num_workers=1, drop_last=True)

Train Images: 2976 Valid Images: 454


### Train

In [9]:
trainer = {}
trainer['model'] = model
trainer['train_data'] = train
trainer['valid_data'] = valid
trainer['loss_fn'] = loss
trainer['metric_fn'] = metric
trainer['optimizer'] = optimizer
trainer['save_path'] = '../../data/model.pt'
trainer['epochs'] = 2
trainer['batch'] = 4
trainer['scheduler'] = schedular

In [ ]:
trainModel(**trainer)

 49% 1452/2976 [02:55<02:55,  8.67it/s, trn_ls=0.91306]